# 第4章 因果推論を実装しよう

#### ・本章では、因果の大きさを推定する、すなわち因果推論を実施する手法として、回帰分析、IPTW法、DR法について解説、実装する .

## 4-1 回帰分析による因果推論の実装

## 回帰モデルの導出

#### ・回帰分析による因果推論では、入力変数を与えたときに出力変数の値を求めるモデルを構築する .

#### ・回帰モデルの入力変数 : バックドアパスを閉じるように因果ダイアグラムをd分離したあとに残っている変数のみを使用 .

#### ・回帰モデルの出力変数 : 因果効果が表れる変数 .

#### ・回帰分析による因果推論を英語では regression adjustment と呼ぶ .

#### ・因果推論したいモデルとして、以下のテレビCM効果の架空事例を想定し、この因果ダイアグラムからテレビCM効果を推定する .

![alt text](pict1.png)

#### ・図4.1.1の年齢を変数$x_1$、性別を変数$x_2$として表し、とある「iさん」の年齢と性別は、$x^i_1$ , $x^i_2$と表すことにする .

#### ・とある「iさん」が[テレビCMを見た]かどうか、すなわち処置を受けたかどうかを$Z^i$と記載する .

#### テレビCMを見た場合は $Z^i$ = 1 となり、$Z^i_1$と記載する .　テレビCMを見ていない場合は $Z^i$ = 0 となり、$Z^i_0$と記載する .

#### この変数$Z$のように、処置を受けたなどを示す変数は"原因変数"とも呼ばれる .

#### ・次に、とある「iさん」の商品の購入量(連続変数)を$Y^i$と記載する .

#### $Z^i$ = 1 の場合は$Y^i_1$と記載し、$Z^i$ = 0 の場合は$Y^i_0$と記載する .

#### $Y$のような、処置による結果が現れる変数は"結果変数"とも呼ばれる .

#### ・図4.1.1では年齢変数$x_1$と性別変数$x_2$は、変数$Z$と変数$Y$の間に疑似相関を生む交絡因子(共通因子)である .　そのためd分離するには変数$x_1$と変数$x_2$を考慮する .

#### これは感覚的には、d分離した後に残っている変数(共変量)を考慮して、結果変数を求めるという操作である .

#### ・この場合、具体的には以下のような2つの式を立てる .

$$
Y^i_1 = f_1(Z^i,x^i_1,x^i_2)
$$
$$
Y^i_0 = f_0(Z^i,x^i_1,x^i_2)
$$

#### ・ここで、本説の例では結果変数$Y$が連続値なので、上記の$f()$には線型回帰を利用する .

#### 結果変数$Y$が離散値(カテゴリー変数)であった場合にはロジスティック回帰を使用する .

#### ・ここで、性別を示す変数$x_2$はカテゴリー変数であり、女性の場合を0、男性の場合を１とする .

#### 年齢を示す変数$x_1$は連続的な整数値となる .

#### ・線型回帰モデルの形で上式を書き直すと、下のようになる .

$$
Y^i_1 = b_{z1}Z^i_1 + b_1x^i_1 + b_2x^i_2 + b_0 = b_z × 1 + b_1x^i_1 + b_2x^i_2 + b_0
$$
$$
Y^i_0 = b_{z0}Z^i_0 + b_1x^i_1 + b_2x^i_2 + b_0 = b_z × 0 + b_1x^i_1 + b_2x^i_2 + b_0
$$

#### ここで、変数$b$は各係数を示し、$b_0$はバイアス項である .　上記において処置の変数$Z$は0か1の2値であり、$b_{z0}$にかかる$Z^i_0$が0より、$b_{z1}$ = $b_{z0}$ = $b_z$ と書いても問題ない .

#### ・その結果、関数$f_0$と関数$f_1$は同一になり、回帰モデルは最終的に以下のようになる .

$$
Y^i = b_zZ^i + b_1x^i_1 + b_2x^i_2 + b_0
$$

#### あとは実際のデータからこのモデルの係数$b_z$,$b_1$,$b_2$,$b_0$を求めればよい .

## 回帰モデルの実装

#### ・本節では、テレビCM効果を確かめる疑似ケースの実装を行う .

#### 回帰モデルを求める際に、Pythonの機械学習ライブラリであるscikit-learnを使用する .

In [12]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

In [13]:
# 使用するパッケージ（ライブラリと関数）を定義
# 標準正規分布の生成用
from numpy.random import *

# グラフの描画用
import matplotlib.pyplot as plt

# SciPy 平均0、分散1に正規化（標準化）関数
import scipy.stats

# シグモイド関数をimport
from scipy.special import expit

# その他
import pandas as pd

#### ・はじめに疑似データを作成する .

#### 年齢変数$x_1$は15歳から75歳の一様分布に従うとする .　性別変数$x_2$は0を女性、1を男性とし、50%の確率で男性か女性とする .

In [14]:
# データ数
num_data = 200

# 年齢
x_1 = randint(15, 76, num_data)  # 15から75歳の一様乱数

# 性別（0を女性、1を男性とします）
x_2 = randint(0, 2, num_data)  # 0か1の乱数

#### ・次にテレビCMを見たかどうかを決める .

#### 年齢$x_1$が高いほど、そして性別$x_2$が男性よりも女性の方がテレビCMを見る確率(処置を受け、$Z^i$が1となる確率)が高いとする .

#### この$Z^i$を作るために、シグモイド関数

$$
sigmoid(x) = 1/(1 + exp(-ax))
$$

#### を利用して、

$$
Z^i_{prob} = sigmoid[x_1 + (1-x_2) × 10-40 + noise^i]
$$

#### を計算する .　$Z^i_{prob}$は0から1の値となり、その割合に応じて確率的にテレビCMを見たかどうか($Z^i$ = 1)を計算する .

#### つまり、$Z^i_{prob}$が0に近いと$Z^i$ = 0になりやすく、$Z^i_{prob}$が1に近いと$Z^i$は1になりやすい .

#### 実装コードは次の通りである .　シグモイド関数の係数$a$は0.1としている .

In [15]:
# ノイズの生成
e_z = randn(num_data)

# シグモイド関数に入れる部分
z_base = x_1 + (1-x_2)*10 - 40 + 5*e_z

# シグモイド関数を計算
z_prob = expit(0.1*z_base)

# テレビCMを見たかどうかの変数（0は見ていない、1は見た）
Z = np.array([])

for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)

#### ・購入量$Y^i$は、

$$
Y^i = -x_1 + 30x_2 + 10Z^i + 80 + noise^i
$$

#### で決まるとする .　年齢$x_1$が大きいほど購入量は減少し、男性($x_2$ = 1)の方が購入量は多く、テレビCMを見ていると($Z^i$ = 1)購入量が増える .

#### ここで、テレビCMの係数が10なので、テレビCMによる購入量への効果は+10が正解となる .　購入量$Y^i$の実装は次の通りである .

In [16]:
# ノイズの生成
e_y = randn(num_data)

Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y

#### ・ここで、各データをまとめた表を作成し、さらにCMを見た人と見ていない人で購入量$Y$などの平均を比較してみる .

In [17]:
# データフレームの作成
df = pd.DataFrame({'年齢': x_1,
                   '性別': x_2,
                   'CMを見た': Z,
                   '購入量': Y,
                   })
# 先頭を表示
df.head() 

,年齢,性別,CMを見た,購入量
0,62,0,1.0,24.464285
1,34,0,0.0,45.693411
2,53,1,1.0,64.998281
3,68,1,1.0,47.186898
4,27,1,0.0,100.114260


In [18]:
# 平均値を比べる
print(df[df["CMを見た"] == 1.0].mean())
print("--------")
print(df[df["CMを見た"] == 0.0].mean())

年齢       55.836066
性別        0.483607
CMを見た     1.000000
購入量      49.711478
dtype: float64
--------
年齢       32.141026
性別        0.692308
CMを見た     0.000000
購入量      68.827143
dtype: float64


#### ・テレビCMを見た人の方が、平均年齢が高く、また性別も女性が多い .

#### しかし、テレビCMを見た人の平均購入量は約49.7に対し、テレビCMを見ていない人の平均購入量は約68.8となり、テレビCMを見ていない人の方が平均購入量が多くなっている .

#### 単純に平均値の差を見ると、テレビCMを見た人の方が、平均購入量が約20も少なくなっている .

#### ・続いて、この作成した模擬データを用いて、テレビCMの効果を因果推論する回帰分析を実行する .

In [19]:
# scikit-learnから線形回帰をimport
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
from sklearn.linear_model import LinearRegression

# 説明変数
X = df[["年齢", "性別", "CMを見た"]]

# 被説明変数（目的変数）
y = df["購入量"]

# 回帰の実施
reg = LinearRegression().fit(X, y)

# 回帰した結果の係数を出力
print("係数：", reg.coef_)

係数： [-0.95817951 32.70149412 10.41327647]


#### ・出力結果から、年齢に対する係数が-0.95、性別に対する係数は32.7、CMを見たことによる購入量の増加の係数は10.4となった .

#### ・模擬データの係数はそれぞれ、-1、30、10であったので、データ生成時の係数と因果推論した係数がほぼ一致している .

#### ・よって、CMを見たことによる購入量の増加は10.4と推定され、「テレビCMを見ると、購入量が平均で10.4増える」という処理の効果が明らかになった .

#### ・最後に平均処置効果ATE、処置群における平均処置効果ATT、対照群における平均処置効果ATUを求める .

#### 今回は変数$Y^i$についての回帰モデルを線形に構築しているので、変数$Z^i$の係数の10.4が因果の効果となり、ATE , ATT , ATUはすべて等しく10.4となる .

#### ・線形な回帰モデルを構築し、入力変数にはd分離したあとで因果ダイアグラムに残っている変数を使用した .

#### 理由として、d分離したあとでも残っている変数はバックドアパスを生むため、バックドアパスを閉じるために考慮する必要があるため .

#### 回帰分析で回帰モデルを構築する際に、これらの変数を入力変数に利用することは、これらの変数からのバックドアパスの存在を閉じることになる .

#### ・今回の結果のように、年齢に対する係数-0.95、性別に対する係数32.7と、共変量が結果変数に与える影響を個別に求めることができ、結果変数に対する原因変数$Z$の影響と共変量の影響を分離できる .

## 回帰モデルの補足

#### ・以下に示す図4.1.3のように、仮に結果変数$Y$にだけ効く別の変数$a$が観測できている場合、d分離する際に変数$a$を残す必要はない .

#### つまり、変数$a$は疑似相関、間接的因果を生み出していないが、変数$Y$の正確な回帰モデルを構築するために回帰モデルの入力変数として考慮する方が正確な因果推論が可能になる .

![alt text](pict2.png)

## 4-2 傾向スコアを用いた逆確率重み付け法(IPTW)の実装

#### ・傾向スコアと呼ばれる指標を使用した、逆確率重み付け法(IPTW : Inverse Probability of Treatment Weighting もしくは IPW)を解説・実装する .

## 傾向スコアとは

#### ・傾向スコア(propensity score) : 処置を受ける確率に類する値、すなわち「処置を受ける傾向」を示す指標 .

#### ・IPTW法 : 測定されたデータに対して、傾向スコアで結果を調整して、因果の効果を推定する手法 .

#### ・はじめに傾向スコアについて解説する .　ここで、調整化公式を再掲する .

$$
P(Y = y|do(Z = z)) = Σ_xP(Y = y|Z = z,X = x)P(X = x)
$$

#### ・この調整化公式の右辺は2つの項、$P(Y = y|Z = z,X = x)$ と $P(X = x)$の掛け算となっていて、取り扱いにくいという嫌な点がある .

#### そこで、これらの項を1つにする .　同時確率と条件付き確率の変換を用いると、

$$
P(Y = y|Z = z,X = x) = P(Y = y,Z = z|X = x)/P(Z = z|X = x)
$$

#### となる .　よって、$P(Y = y|Z = z,X = x)P(X = x)$は、

$$
P(Y = y,Z = z|X = x)P(X = x)/P(Z = z|X = x)
$$

#### となる .　そして、分子を$X$について条件付き確率から同時確率へと変換すると、

$$
P(Y = y,Z = z|X = x)P(X = x) = P(Y = y,Z = z,X = x)
$$

#### となる .　よって、$P(Y = y|Z = z,X = x)P(X = x)$は、

$$
P(Y = y,Z = z,X = x)/P(Z =z|X = x)
$$

#### となる .　以上の式変形を用いて、調整化公式を変形すると、

$$
P(Y = y|do(Z = z)) = Σ_xP(Y = y,Z = z,X = x)/P(Z =z|X = x) = Σ_xP(x,y,z)/P(z|x)
$$

#### と求められる .

#### ・ここまでの式変形で、変数$X$の組み合わせ計算に掛け算がなくなったことが分かる .

#### 上式の分子$P(x,y,z)$は、とある$(X = x,Y = y,Z = z)$のサンプルの割合を示す .

#### 分母の$P(z|x)$は変数$X$が値$x$のもとで変数$Z$が値$z$となる確率であり、これを"傾向スコア(propensity score)"と呼ぶ .

### 変数$X$はテレビCMの例では年齢や性別であったので、傾向スコアとは、とある人の属性情報に応じたテレビCMを見る確率、すなわち処置を受ける確率を意味する .

## 傾向スコアの実装

#### ・傾向スコアを求める確率式は変数$Z$の値の確率値が求まるモデルであれば何でもよいが、基本的にはロジスティック回帰が使われる .

#### ・回帰分析でもシグモイド関数を用いて、とある「iさん」がテレビCMを見る確率$Z^i_{prob}$を

$$
Z^i_{prob} = sigmoid[x_1 + (1-x_2) × 10-40 + noise^i]
$$

#### として作成した .　ここで、シグモイド関数は

$$
sigmoid(x) = 1/(1 + exp(-ax))
$$

#### であり、回帰分析では$a$ = 0.1として計算したデータを作成した .

#### ・よって傾向スコアをロジスティック回帰で求めるとは、疑似データを作成したシグモイド関数のモデル係数を求めることになる .　式で書くと、

$$
\hat{Z}^{i}_{prob} = \text{sigmoid}\left\{\beta_1 x_1 + \beta_2 x_2 + \alpha\right\}
$$


#### の、係数$\beta_1$ , $\beta_2$ , $\alpha$ をデータから求めることになる .　実装コードは以下の通りになる .

In [20]:
# scikit-learnからロジスティク回帰をimport
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
from sklearn.linear_model import LogisticRegression

# 説明変数
X = df[["年齢", "性別"]]

# 被説明変数（目的変数）
Z = df["CMを見た"]

# 回帰の実施
reg = LogisticRegression().fit(X,Z)

# 回帰した結果の係数を出力
print("係数beta：", reg.coef_)
print("係数alpha：", reg.intercept_)

係数beta： [[ 0.10562765 -1.38263933]]
係数alpha： [-3.37146523]


#### ・求まった結果は$\beta_1$ = 0.1 , $\beta_2$ = -1.4 , $\alpha$ = -3.4 です .　正しい答えである 0.1 , -1 , -3にほぼ近い係数が求まっており、傾向スコア$P(z|x)$をうまくモデル化できた .

#### ・続いて、それぞれの人の傾向スコア$\hat{Z}^{i}_{prob}$を求める .

In [21]:
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])  # 5人ほどの結果を見てみる
print("----")
print(Z[0:5])  # 5人ほどの正解

[[0.04002323 0.95997677]
 [0.44525168 0.55474832]
 [0.30065918 0.69934082]
 [0.08101946 0.91898054]
 [0.87013558 0.12986442]]
----
0    1.0
1    0.0
2    1.0
3    1.0
4    0.0
Name: CMを見た, dtype: float64


#### ・この結果を確認すると、傾向スコアの1になる確率が大きいと(出力の2列目の値が大きいと)、実際にテレビCMを見ている傾向にある .

#### ・各人の傾向スコアが求まったので、最後にATEを求める .　平均処置効果ATEは

$$
ATE = E(Y_1) - E(Y_0) = E(Y_1|do(Z = 1)) - E(Y_0|do(Z = 0))
$$

#### であり、調整化公式を用いて変形すると、

$$
ATE = Σ_xE(Y|Z = 1,X = x)P(X = x) - Σ_xP(Y|Z = 0,X = x)P(X = x)
$$

#### となる .　ここで、本節で導入した式変形により、

$$
ATE = Σ_xP(Y,Z = 1,X = x)/P(Z = 1|X = x) - Σ_xP(Y,Z = 0,X = x)P(Z = 0|X = x)
$$

#### となる .　ここで、分母にある$P(Z = 1|X = x)$や$P(Z = 0|X = x)$が傾向スコアである .

#### ・そして、$Y$が今回のように離散値ではなく連続値の場合は、

$$
ATE = \frac{1}{N} \sum_{i=1}^N \frac{y_i}{P(Z = 1 \mid X = x_i)} Z_i 
- \frac{1}{N} \sum_{i=1}^N \frac{y_i}{P(Z = 0 \mid X = x_i)} (1 - Z_i)
$$

#### となる .　この数式計算を実装すると次の通りになる .

In [22]:
ATE_i = Y/Z_pre[:, 1]*Z - Y/Z_pre[:, 0]*(1-Z)
ATE = 1/len(Y)*ATE_i.sum()
print("推定したATE", ATE)

推定したATE 8.84747681085544


#### ・回帰分析にて購入量$Y^i$は、

$$
Y^i = -x_1 + 30x_2 + 10Z^i + 80 + noise^i
$$

#### としており、テレビCMを見ると購入量が10増えるモデルだった .　ATEの推定結果も約8.8となっており、テレビCMによる効果(因果の大きさ)がうまく推定されている .

#### 今はデータ数が200個だけなので、データ数を増やせば、因果推論の結果は10に近づき、推定精度が向上する .

### ・IPTW法の注意点として、傾向スコアで割り算をしているので、傾向スコアが0に近いと計算が不安定になる .

## 4-3 Doubly Robust法(DR法)による因果推論の実装

## IPTW法の欠点とDR法

#### ・先ほど解説したIPTW法ですが、平均処置効果ATEの計算式は次の通りでした .

$$
ATE = \frac{1}{N} \sum_{i=1}^N \frac{y_i}{P(Z = 1 \mid X = x_i)} Z_i 
- \frac{1}{N} \sum_{i=1}^N \frac{y_i}{P(Z = 0 \mid X = x_i)} (1 - Z_i)
$$

#### この式では、とある「iさん」がテレビCMを見ていない場合には$Z_i$ = 0、そして、$(1-Z_i)$ = 1である .

#### つまり、テレビCMを見ていない人のデータの場合、2項あるうちの後半の項しか使用されていない .

#### ・とはいえ、ここでの「iさん」がテレビCMを見た場合の$Y^i_1$の値は反実仮想であり、不明である .

#### そこで、回帰分析で回帰モデルを構築し、反実仮想である$Y^i_1$の推定値$\hat{Y}^{i}$を計算することにする . 

#### そして、回帰分析から求めた反実仮想の推定値とIPTW法と組み合わせるため、とある「iさん」の処置効果TEの前半の項を、

$$
\frac{y_i}{P(Z=1 \mid X = x_i)} Z_i
$$

#### ではなく、

$$
\frac{y_i}{P(Z=1 \mid X = x_i)} Z_i + \left(1 - \frac{Z_i}{P(Z=1 \mid X = x_i)} \right) \hat{Y}_1^i
$$

#### として考えることにする .　この式では$Z_i$ = 0 の場合、

$$
\left(1 - \frac{Z_i}{P(Z=1 \mid X = x_i)} \right) \hat{Y}_1^i = \hat{Y}_1^i
$$

#### となり、反実仮想である$E(Y_1|do(Z=1))$の値が$\hat{Y}_1^i$となる .　このように表現することで、IPTW法では考慮できていなかった各人の反実仮想を計算に加えることができる .

#### ・同様に後半の項

$$
\frac{y_i}{P(Z=0 \mid X = x_i)} (1 - Z_i)
$$

#### に関しては、

$$
\frac{y_i}{P(Z=0 \mid X = x_i)} (1 - Z_i) + \left(1 - \frac{(1-Z_i)}{P(Z=0 \mid X = x_i)} \right) \hat{Y}_0^i
$$

#### とする .　すると、テレビCMを見た人の場合、$Z_i$ = 1より、後半の項は回帰分析で求める反実仮想$\hat{Y}_0^i$となる .

### ・このように、IPTW法での推定と回帰分析での推定を組み合わせて因果の効果を推定する方法を、"Doubly Robust Estimation (DR法)" と呼ぶ .

## DR法の実装

#### ・データは4.1 , 4.2節と同じテレビCMの効果を推定する .

#### ・回帰分析の回帰モデルの構築、傾向スコアを推定するロジスティック回帰モデルの構築は、それぞれ4.1 , 4.2節と同じになる .

#### ・はじめに線形回帰モデルの構築を行う .　そして、テレビCMを見た場合の購入量$\hat{Y}_1^i$ 、 見ていない場合の購入量$\hat{Y}_0^i$を計算する .

In [23]:
# scikit-learnから線形回帰をimport
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
from sklearn.linear_model import LinearRegression

# 説明変数
X = df[["年齢", "性別", "CMを見た"]]

# 被説明変数（目的変数）
y = df["購入量"]

# 回帰の実施
reg2 = LinearRegression().fit(X, y)

# Z=0の場合
X_0 = X.copy()
X_0["CMを見た"] = 0
Y_0 = reg2.predict(X_0)

# Z=1の場合
X_1 = X.copy()
X_1["CMを見た"] = 1
Y_1 = reg2.predict(X_1)

#### ・続いて、傾向スコアを求めるロジスティック回帰モデルを構築する .

In [24]:
# scikit-learnからロジスティク回帰をimport
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
from sklearn.linear_model import LogisticRegression

# 説明変数
X = df[["年齢", "性別"]]

# 被説明変数（目的変数）
Z = df["CMを見た"]

# 回帰の実施
reg = LogisticRegression().fit(X, Z)

# 傾向スコアを求める
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])  # 5人ほどの結果を見てみる

[[0.04002323 0.95997677]
 [0.44525168 0.55474832]
 [0.30065918 0.69934082]
 [0.08101946 0.91898054]
 [0.87013558 0.12986442]]


#### ・最後に、ATEの推定を実装する .　各人のTE(ITE)はDR法では、

$$
\frac{y_i}{P(Z=1 \mid X = x_i)} Z_i + \left(1 - \frac{Z_i}{P(Z=1 \mid X = x_i)} \right) \hat{Y}_1^i
$$

#### と

$$
\frac{y_i}{P(Z=0 \mid X = x_i)} (1 - Z_i) + \left(1 - \frac{(1-Z_i)}{P(Z=0 \mid X = x_i)} \right) \hat{Y}_0^i
$$

#### の差なので、実装は次の通りになる .

In [25]:
ATE_1_i = Y/Z_pre[:, 1]*Z + (1-Z/Z_pre[:, 1])*Y_1
ATE_0_i = Y/Z_pre[:, 0]*(1-Z) + (1-(1-Z)/Z_pre[:, 0])*Y_0
ATE = 1/len(Y)*(ATE_1_i-ATE_0_i).sum()
print("推定したATE", ATE)

推定したATE 9.752775054248458


#### ・推定したATEは約9.8となった .

#### ・4.1節での回帰分析でのテレビCMによる購入量の増加の係数は約10.4、IPTW法での推定結果は約8.8、そしてDR法では約9.8となった .

#### ・データで作成した際の答えであるテレビCMの効果は10であったので、DR法によって実際に推定結果が良くなることが確かめられた .

## 次回

#### ・本章で紹介した手法は、傾向スコアも購入量も、年齢や性別、CM効果といった各変数が線形に影響しているという前提がある .

#### 「年齢によって効果が変化する」、「性別によって効果が異なる」などのように、非線形かつ相互作用があるケースでは、機械学習を用いた因果推論手法が近年提案されているので、それらの手法を第5章では解説、実装する .